In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("thanksgiving.csv", encoding="Latin-1")

In [4]:
data.head(2)

,RespondentID,Do you celebrate Thanksgiving?,What is typically the main dish at your Thanksgiving dinner?,What is typically the main dish at your Thanksgiving dinner? - Other (please specify),How is the main dish typically cooked?,How is the main dish typically cooked? - Other (please specify),What kind of stuffing/dressing do you typically have?,What kind of stuffing/dressing do you typically have? - Other (please specify),What type of cranberry saucedo you typically have?,What type of cranberry saucedo you typically have? - Other (please specify),...,Have you ever tried to meet up with hometown friends on Thanksgiving night?,"Have you ever attended a ""Friendsgiving?""",Will you shop any Black Friday sales on Thanksgiving Day?,Do you work in retail?,Will you employer make you work on Black Friday?,How would you describe where you live?,Age,What is your gender?,How much total combined money did all members of your HOUSEHOLD earn last year?,US Region
0,4337954960,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,None,NaN,...,Yes,No,No,No,NaN,Suburban,18 - 29,Male,"$75,000 to $99,999",Middle Atlantic
1,4337951949,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Other (please specify),Homemade cranberry gelatin ring,...,No,No,Yes,No,NaN,Rural,18 - 29,Female,"$50,000 to $74,999",East South Central


In [5]:
data.columns

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?',
       'How is the main dish typically cooked? - Other (please specify)',
       'What kind of stuffing/dressing do you typically have?',
       'What kind of stuffing/dressing do you typically have? - Other (please specify)',
       'What type of cranberry saucedo you typically have?',
       'What type of cranberry saucedo you typically have? - Other (please specify)',
       'Do you typically have gravy?',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Brussel sprouts',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Carrots',
       'Which of these side dishes aretypically served

In [6]:
data["Do you celebrate Thanksgiving?"].value_counts()

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

In [7]:
data = data[data["Do you celebrate Thanksgiving?"] == "Yes"]

In [8]:
data["What is typically the main dish at your Thanksgiving dinner?"].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

In [9]:
data[data["What is typically the main dish at your Thanksgiving dinner?"] == 
     "Tofurkey"]["Do you typically have gravy?"]

4      Yes
33     Yes
69      No
72      No
77     Yes
145    Yes
175    Yes
218     No
243    Yes
275     No
393    Yes
399    Yes
571    Yes
594    Yes
628     No
774     No
820     No
837    Yes
860     No
953    Yes
Name: Do you typically have gravy?, dtype: object

In [10]:
apple_is_null = pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple"])
pumpkin_is_null = pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin"])
pecan_is_null = pd.isnull(data ["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan"])

ate_pies = apple_is_null & pumpkin_is_null & pecan_is_null

ate_pies.value_counts()

False    876
True     104
dtype: int64

In [11]:
def convert_age(column):
    if pd.isnull(column):
        return None
    else:
        value = column.split()
        try:
            return int(value[0])
        except ValueError:
            return int(value[0][:2])
        
data["int_age"] = data["Age"].apply(convert_age)

In [12]:
data["int_age"].describe()

count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%       30.000000
50%       45.000000
75%       60.000000
max       60.000000
Name: int_age, dtype: float64

There were 947 participants that answered the age question.  The mean age is about 40 and median age is 45, however we lumped every person from an age range into one age, so these numbers are not a true depiction of these values.  The max value is definitely misleading as the category was 60+ so there were definitely older participants than 60.

In [13]:
def convert_income(column):
    if pd.isnull(column):
        return None
    else:
        income = column.split()
        if income[0] == 'Prefer':
            return None
        else:
            no_symbols = ''
            for s in income[0]:
                if s not in ["$",","]:
                    no_symbols += s
            return int(no_symbols)

In [14]:
data["int_income"] = data["How much total combined money did all members of your HOUSEHOLD earn last year?"].apply(convert_income)

In [15]:
data["int_income"].describe()

count       829.000000
mean      75965.018094
std       59068.636748
min           0.000000
25%       25000.000000
50%       75000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64

Only 829 identified their hoselhold income level.  The mean was 75965 and the median was 75,000.  These numbers are likely low given that we used the minimum value for each income range.  Again the max of the range is misleading as the top income could be any value higher than 200k.  The standard deviation is quite high, so there is a lot of variation in incomes between the participants.  The interquartile range from 25000-100000 is again probably a little low due to the values used, but seems like a reasonable representation of most of the population.

In [16]:
data[data["int_income"] < 150000]["How far will you travel for Thanksgiving?"].value_counts()

Thanksgiving is happening at my home--I won't travel at all                         281
Thanksgiving is local--it will take place in the town I live in                     203
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    150
Thanksgiving is out of town and far away--I have to drive several hours or fly       55
Name: How far will you travel for Thanksgiving?, dtype: int64

In [17]:
data[data["int_income"] > 150000]["How far will you travel for Thanksgiving?"].value_counts()

Thanksgiving is happening at my home--I won't travel at all                         49
Thanksgiving is local--it will take place in the town I live in                     25
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    16
Thanksgiving is out of town and far away--I have to drive several hours or fly      12
Name: How far will you travel for Thanksgiving?, dtype: int64

At both income levels, a large proportion have Thanksgiving at home.  Above 150k is more likely to have Thanksgiving at home or travel a long distance, below 150k is more likely to travel shorter distances for Thanksgiving. 

In [18]:
data.pivot_table(index = "Have you ever tried to meet up with hometown friends on Thanksgiving night?",
                columns = 'Have you ever attended a "Friendsgiving?"', values="int_age")

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,42.283702,37.010526
Yes,41.475410,33.976744


In [19]:
data.pivot_table(index = "Have you ever tried to meet up with hometown friends on Thanksgiving night?",
                columns = 'Have you ever attended a "Friendsgiving?"', values="int_income")

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,78914.549654,72894.736842
Yes,78750.000000,66019.736842


These questions do seem skewed toward younger people, especially the question on "Friendsgiving".  I believe "Friendsgiving" is a much newer term, a question like "Have you ever celebrated Thanksgiving with a group of friends?" might speak to a broader population.  Also a lower income trends toward "Friendsgiving", which is indicative of both a younger generation as well as those without the means to travel to join their family.